In [16]:
import pandas as pd
import os
from langchain_community.graphs import Neo4jGraph
from tqdm import tqdm
import numpy as np
import re

pd.set_option('display.max_columns', 500)


In [17]:
raw_requests = pd.read_csv('rules_data.csv')
raw_requests.head(1)

C:\Users\vism\AppData\Local\Temp\ipykernel_3420\1714904036.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_requests = pd.read_csv('rules_data.csv')


,timestamp [UTC],id,source,name,url,success,resultCode,duration,performanceBucket,itemType,customDimensions,customMeasurements,operation_Name,operation_Id,operation_ParentId,operation_SyntheticSource,session_Id,user_Id,user_AuthenticatedId,user_AccountId,application_Version,client_Type,client_Model,client_OS,client_IP,client_City,client_StateOrProvince,client_CountryOrRegion,client_Browser,cloud_RoleName,cloud_RoleInstance,appId,appName,iKey,sdkVersion,itemId,itemCount,_ResourceId
0,"12/7/2024, 2:12:47.609 pm",cc286184c6d9b7c9,cid-v1:443108a7-2c63-4025-90d6-483f3023e351,POST Rules/Rules,http://rules-api-service.rules.svc.cluster.loc...,True,200,1424.3322,1sec-3sec,request,"{""X-Parent-Source"":""V4"",""CompanyContext"":""CDNV...",NaN,POST Rules/Rules,8ff8696695aa73588ac454809741e2ea,73c2f42cf4fa3ee9,NaN,NaN,NaN,NaN,NaN,6.0.32,PC,NaN,NaN,0.0.0.0,NaN,NaN,NaN,NaN,rules-api,rules-api-deployment-7548b55df9-5q74p,f3c32fb8-ed5e-4045-b519-12e185ca365d,/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e...,db9cf7b1-a381-4315-ae3e-fdcf9a827987,ai-k8s:6.1.1.0,e8ecf2ac-4058-11ef-94c0-00224809857f,1,/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e...


In [18]:
# select only required columns
df_requests = raw_requests[['id', 'timestamp [UTC]', 'name', 'url', 'duration', 'customDimensions', 'operation_Id', 'operation_ParentId']]
# measure at APIM level
df_requests = df_requests[df_requests['name'] == 'POST Rules/Rules']
df_requests.head(1)

,id,timestamp [UTC],name,url,duration,customDimensions,operation_Id,operation_ParentId
0,cc286184c6d9b7c9,"12/7/2024, 2:12:47.609 pm",POST Rules/Rules,http://rules-api-service.rules.svc.cluster.loc...,1424.3322,"{""X-Parent-Source"":""V4"",""CompanyContext"":""CDNV...",8ff8696695aa73588ac454809741e2ea,73c2f42cf4fa3ee9


In [19]:
df_requests['duration'].describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99])

count      4420.000000
mean       4425.952405
std       12562.126106
min         125.303000
50%         544.901550
75%        1317.375575
90%       10225.018560
95%       24079.596710
99%       70053.827974
max      110924.389400
Name: duration, dtype: float64

In [20]:
operation_ids = df_requests[df_requests['duration'] > 1000]['operation_Id'].unique()
print(f'Number of unique operation ids: {len(operation_ids)}')
op_id = operation_ids[0]
print(f'Example operation id: {op_id}')

Number of unique operation ids: 1346
Example operation id: 8ff8696695aa73588ac454809741e2ea


In [62]:
raw_dependencies = pd.read_csv('query_data (33).csv')
raw_dependencies.head(1)
print(f'Number of unique operation ids: {len(raw_dependencies["operation_Id"].unique())}')

Number of unique operation ids: 380


In [64]:
t = []
df_dep = pd.DataFrame()
# for op_id in operation_ids.take(1):
service_dep_graph = raw_dependencies[raw_dependencies['operation_Id'] == op_id].sort_values('timestamp [UTC]', ascending=True)
print(len(service_dep_graph))
service_dep_graph.head()

68


,timestamp [UTC],id,operation_Id,itemType,operation_ParentId,operation_Name,iKey,duration,type,name,cloud_RoleInstance,o_count,cloud_RoleName
122,"12/7/2024, 2:12:47.188 pm",72d50ef82523b6dd,8ff8696695aa73588ac454809741e2ea,request,|c271aa9f-465a15e0b465d34e.c271aaa1_,POST Timesheet/SubmitShiftEntry,545301b3-f114-4865-9afa-0ca19c1d142b,1868.0681,NaN,POST Timesheet/SubmitShiftEntry,v4-api-deployment-897595574-ksg78,68,v4-api
114,"12/7/2024, 2:12:47.263 pm",e5bc78812019f353,8ff8696695aa73588ac454809741e2ea,dependency,72d50ef82523b6dd,POST Timesheet/SubmitShiftEntry,545301b3-f114-4865-9afa-0ca19c1d142b,34.0293,Http (tracked component),POST /api/readallcentralrecords-func,v4-api-deployment-897595574-ksg78,68,v4-api
101,"12/7/2024, 2:12:47.271 pm",171e6a3358258d98,8ff8696695aa73588ac454809741e2ea,request,e5bc78812019f353,readallcentralrecords-func,db9cf7b1-a381-4315-ae3e-fdcf9a827987,22.0563,NaN,readallcentralrecords-func,b06e57d8b1788d9a42542c457af7292942644db9c4e4e5...,68,core-func-shift-perf
113,"12/7/2024, 2:12:47.306 pm",c0af2c3968b30c11,8ff8696695aa73588ac454809741e2ea,dependency,72d50ef82523b6dd,POST Timesheet/SubmitShiftEntry,545301b3-f114-4865-9afa-0ca19c1d142b,35.3667,Http (tracked component),POST /api/updatecentralrecord-func,v4-api-deployment-897595574-ksg78,68,v4-api
100,"12/7/2024, 2:12:47.308 pm",09bfd5ea5809c919,8ff8696695aa73588ac454809741e2ea,request,c0af2c3968b30c11,updatecentralrecord-func,db9cf7b1-a381-4315-ae3e-fdcf9a827987,29.6672,NaN,updatecentralrecord-func,b06e57d8b1788d9a42542c457af7292942644db9c4e4e5...,68,core-func-shift-perf
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,"12/7/2024, 2:12:48.658 pm",101ef37a45739b6b,8ff8696695aa73588ac454809741e2ea,dependency,099f6312827883be,POST PayRate/GetPayRate,db9cf7b1-a381-4315-ae3e-fdcf9a827987,353.8294,HTTP,POST /660a9f76-9ac1-4994-afd2-45b6747006b3/oau...,payroll-api-deployment-5dfd85c476-whbx2,68,payroll-api
94,"12/7/2024, 2:12:48.672 pm",1fb6d6947137e812,8ff8696695aa73588ac454809741e2ea,request,36ec5e41417cf150,readallcentralrecords-multipleparticipant-func,db9cf7b1-a381-4315-ae3e-fdcf9a827987,19.0265,NaN,readallcentralrecords-multipleparticipant-func,62231f4a8d0b0c0d9e2ea445516fe9bb14f36ceec9b723...,68,core-func-shift-perf
121,"12/7/2024, 2:12:48.697 pm",7dd17bb947d313cf,8ff8696695aa73588ac454809741e2ea,dependency,d7f42d34650a0595,POST Rules/GetPayrollRulesData,db9cf7b1-a381-4315-ae3e-fdcf9a827987,195.2058,HTTP,POST /660a9f76-9ac1-4994-afd2-45b6747006b3/oau...,payroll-api-deployment-5dfd85c476-lcwwv,68,payroll-api
120,"12/7/2024, 2:12:48.895 pm",724b4cb182c94f0098007fb9a5b89a07,8ff8696695aa73588ac454809741e2ea,dependency,f60cc08d2b4a16ad,POST Rules/GetPayrollRulesData,db9cf7b1-a381-4315-ae3e-fdcf9a827987,2.8843,SQL,"SQL: tcp:u2-sql-perf.database.windows.net,1433...",payroll-api-deployment-5dfd85c476-lcwwv,68,payroll-api


In [65]:
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Password@123"
graph = Neo4jGraph()

In [80]:
# clear the graph
graph.query('MATCH (n) DETACH DELETE n')

[]

In [81]:
# add nodes
# TO DO: batch insert
nodes = service_dep_graph['cloud_RoleName'].unique()
print(f'Number of nodes: {len(nodes)}')
nodes = list({'id': node} for node in nodes)
nodes.append({'id': 'ROOT'})
for node in nodes:
  query = f"CREATE (a:Node {{id: '{node['id']}'}})"
  graph.query(query)

Number of nodes: 8


In [82]:
# add edges
counter = 0
for index, row in service_dep_graph.iterrows():
  # get the node
  cloud_RoleInstance = row['cloud_RoleInstance']
  id = row['id']
  operation_ParentId = row['operation_ParentId']
  duration = row['duration']
  itemType = row['itemType']
  operation_Name = row['operation_Name']
  # find the parent node
  parents = service_dep_graph[service_dep_graph['id'] == operation_ParentId]['cloud_RoleName'][:1]
  parent_node = None
  if len(parents) > 0:
    parent_node = parents.values[0]
  else:
    parent_node = "ROOT"
  # find the child node
  child_node = service_dep_graph[service_dep_graph['id'] == id]['cloud_RoleName'][:1].values[0]
  # create edge with id as child and operation_ParentId as parent
  if parent_node and child_node:
    query = f"MATCH (a:Node {{id: '{parent_node}'}}), (b:Node {{id: '{child_node}'}}) CREATE (a)-[r:DEPENDS_ON {{duration: {duration}, itemType: '{itemType}', counter: {counter}, operation_Name: '{operation_Name}', cloud_roleInstance: '{cloud_RoleInstance}'}}]->(b) RETURN r"
    result = graph.query(query)
    if len(result) == 0:
      print(f'Error: {query}')
    # print(query)
    counter += 1
print(f'Number of edges: {counter}')

Number of edges: 68


In [83]:
# validation

# get edges
query = "MATCH (n:Node)-[r:DEPENDS_ON]->(m:Node) RETURN n.id, r.duration, m.id"
result = graph.query(query)
print(result[0])

# get total duration
query = "MATCH (n:Node)-[r:DEPENDS_ON]->(m:Node) RETURN sum(r.duration)"
result = graph.query(query)
print(f"Total Duration: {result[0]}")

# order by counter
query = "MATCH (n:Node)-[r:DEPENDS_ON]->(m:Node) RETURN n.id, r.duration, r.counter, m.id, r.operation_Name ORDER BY r.counter asc"
result = pd.DataFrame.from_records(graph.query(query))
result.head(10)

{'n.id': 'v4-api', 'r.duration': 1437.8613, 'm.id': 'v4-api'}
Total Duration: {'sum(r.duration)': 13523.023400000004}


,n.id,r.duration,r.counter,m.id,r.operation_Name
0,ROOT,1868.0681,0,v4-api,POST Timesheet/SubmitShiftEntry
1,v4-api,34.0293,1,v4-api,POST Timesheet/SubmitShiftEntry
2,v4-api,22.0563,2,core-func-shift-perf,readallcentralrecords-func
3,v4-api,35.3667,3,v4-api,POST Timesheet/SubmitShiftEntry
4,v4-api,29.6672,4,core-func-shift-perf,updatecentralrecord-func
5,v4-api,26.3562,5,v4-api,POST Timesheet/SubmitShiftEntry
6,v4-api,18.8664,6,core-func-shift-perf,readallcentralrecords-func
7,v4-api,27.8329,7,v4-api,POST Timesheet/SubmitShiftEntry
8,v4-api,16.8676,8,core-func-shift-perf,readcentralrecord-func
9,ROOT,137.2139,9,v4-api,POST Timesheet/SubmitShiftEntry


In [89]:
# get longest path by duration attribute in the edge, excluding root
query = "MATCH (n:Node)-[r:DEPENDS_ON]->(m:Node) where n.id <> 'ROOT' OR m.id <> 'ROOT' RETURN n.id, r.duration, r.operation_Name, m.id ORDER BY r.duration DESC LIMIT 3"
result = graph.query(query)
for r in result[1:]:
    # update the edge with a new attribute
    query = f"MATCH (n:Node)-[r:DEPENDS_ON]->(m:Node) WHERE n.id = '{r['n.id']}' AND m.id = '{r['m.id']}' SET r.longest_path = true"
    graph.query(query)
pd.DataFrame.from_records(result).head(10)

,n.id,r.duration,r.operation_Name,m.id
0,ROOT,1868.0681,POST Timesheet/SubmitShiftEntry,v4-api
1,v4-api,1437.8613,POST Timesheet/SubmitShiftEntry,v4-api
2,v4-api,1424.3322,POST Rules/Rules,rules-api
